In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
import sys
import os.path as osp
sys.path.insert(1, osp.abspath('..'))

import torch
import yaml
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader, DataLoader
from modules.inference.inference_module import InferenceModule, InferenceEngineAbstract
from models.mcmt.rgnn import MOTMPNet
from modules.torch_dataset.sequence_graph_dataset import SequenceGraphDataset

In [3]:
with open("/Users/hectormejia/Documents/TFM/mcmt_gnn/config/training_rgcnn.yml", "r") as config_file:
    config = yaml.safe_load(config_file)
    gnn_arch = config["gnn_arch"]

model_state = torch.load("/Users/hectormejia/Documents/TFM/mcmt_gnn/results/checkpoints/checkpoint_100.pt")
model = MOTMPNet(gnn_arch)
model.load_state_dict(model_state['model'])


<All keys matched successfully>

In [4]:
data = torch.load("/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC22/graphs/sequence_graph_S02.pt")
inf_module = InferenceModule(model, num_cameras=4)
preds = inf_module(data)

torch.Size([2, 75227])


In [5]:
preds.sum()

tensor(557)

In [ ]:
val_dataset = SequenceGraphDataset(sequence_path_prefix="/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC22", 
                                sequence_names=['S02'], 
                                annotations_filename='gt.txt',
                                transform=T.ToUndirected())
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)#, num_objects_per_graph=10)

In [51]:
class InferenceEngine(InferenceEngineAbstract):
    def setup_validation_step(self):
        def validation_step(engine, batch):
            return self.model(batch), batch.edge_labels
        return validation_step

In [52]:
InferenceEngine(inf_module, val_dataloader, "results",["Accuracy", "Precision", "Recall"]).run_validation()

torch.Size([2, 150454])
tensor([0, 0, 0,  ..., 0, 0, 1])
Inference Results - Accuracy: 0.9949486221702314 - Precision: 0.6386946386946387 - Recall: 0.5490981963927856 - 


/Users/hectormejia/Documents/TFM/mcmt_gnn/modules/inference/postprocessing.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  whole_edges_prediction = split_clusters(G, id_pred, whole_edges_prediction, preds_prob[:,1], num_cameras, data.num_nodes)
